In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv as csv
import random
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

random.seed(42)

In [ ]:
def fillTableWOint(table,df):
    with open(df, 'r') as f:
        reader = csv.reader(f, delimiter=" ")
        row = -1
        for i in reader:
            row +=1
            
            for o in i:
                data = o.split(":")
                
                if(int(data[0]) == 0 or int(data[0])==1):
                    table[row, 0] = int(data[0])
                else:
                    table[row, int(data[0])] = data[1]

    return table

def createNPYWOint(input):
    print(f"{input}")
    table = newTable(input)
    fillTableWOint(table,input)
    print("Done")
    return table

def createNPY(input):
    print(f"{input}")
    table = newTable(input)
    fillTable(table,input)
    print("Done")
    return table

def fillTable(table,df):
    with open(df, 'r') as f:
        reader = csv.reader(f, delimiter=" ")
        row = -1
        for i in reader:
            row +=1
            
            for o in i:
                data = o.split(":")
                
                if(int(data[0]) == 0 or int(data[0])==1):
                    table[row, 0] = int(data[0])
                else:
                    table[row, int(data[0])] = int(data[1])

    return table

def newTable(data):
    max = 0
    count = 0
    
    with open(data, 'r', newline ='') as f:
        reader = csv.reader(f, delimiter=' ')
        for line in reader:
            count +=1
            
            for i in line:
                index = int(i.split(":")[0])
                
                if index > max:
                    max = index
    return np.zeros((count, max+1))

In [ ]:
def create_weights(length):
  w = np.zeros(length)
  for i in range(length):
    w[i] = random.uniform(-0.01,0.01)
  return w

In [ ]:
def create_predictions(w,dataset):
  append = np.ones((dataset.shape[0],1))
  dataset = np.concatenate((dataset,append),axis = 1)
  features = dataset[:,1:]
  #TODO iterate through features and weights to get predictions
  predictions = np.array([])
  for i in range(features.shape[0]):
    
    prediction = predict(features[i],w)
    if prediction > 0:
        predictions = np.append(predictions,1)
    else:
      predictions = np.append(predictions,0)
    #predictions = np.append(predictions, yp)
  return predictions 

In [ ]:
def predict(x,w):
  return x.dot(w) 

In [ ]:
def accuracy(dataset, predictions):
  data = np.copy(dataset)
  
  labels = data[:,0:1]
  correct = 0

  for i in range(len(labels)):
    if labels[i] == predictions[i]:
      correct += 1

  return correct / float(len(labels)) * 100.0

In [ ]:
def sub_gradient(dataset,epochs,lr,C):
  
  #create weight
  w = create_weights(dataset.shape[1])
  
  append = np.ones((dataset.shape[0],1))
  dataset = np.concatenate((dataset,append),axis = 1)

  for epoch in range(1,epochs):
    
    if((epoch/epochs)==0.25):
        print("1/4 way done")
    elif ((epoch/epochs)==0.50):
        print("1/2 way done")
    elif ((epoch/epochs)==0.75):
        print("3/4 way done")
    
    
    # shuffling data and then choose
    dataset_shuffle = np.copy(dataset)
    np.random.shuffle(dataset_shuffle)
    features = dataset_shuffle[:,1:]
    labels = dataset_shuffle[:,0:1]
    for i in range(features.shape[0]):
      rate = learning_rate(epochs,lr)
      
      if (labels[i,0] == 0):
        y = -1
      else:
        y = 1
    
      if (y * np.dot(w.T,features[i])  <= 1):
        w =(1-rate)*w+(rate*y*C*features[i])
      else:
        w = (1-rate)*w
    #print(epoch)
  #print("done")
  return w

def learning_rate(epochs,lr):
  y_t = lr/(1+epochs)
  return y_t

In [ ]:
#test sub gradient 
input_file_train = '/kaggle/input/uofu-ml-fall-2020/project_data/data/bag-of-words/bow.train.libsvm'
input_file_test = '/kaggle/input/uofu-ml-fall-2020/project_data/data/bag-of-words/bow.test.libsvm'
input_file_eval = '/kaggle/input/uofu-ml-fall-2020/project_data/data/bag-of-words/bow.eval.anon.libsvm'

train = createNPY(input_file_train)
test = createNPY(input_file_test)
evalTable = createNPY(input_file_eval)
pd.DataFrame(train).to_csv("/kaggle/working/train.csv", index=None)
pd.DataFrame(test).to_csv("/kaggle/working/test.csv", index=None)
pd.DataFrame(evalTable).to_csv("/kaggle/working/evalTable.csv", index=None)

train = pd.read_csv("/kaggle/working/train.csv",dtype=int,header = None)
train = train.to_numpy()

test = pd.read_csv("/kaggle/working/test.csv",dtype=int,header = None)
test = test.to_numpy()

evalTable = pd.read_csv("/kaggle/working/evalTable.csv",dtype=int,header = None)
evalTable = evalTable.to_numpy()
print("Done saving csvs")

In [ ]:
# print("Testing Sub-Gradient SVM...")
# y_o=[1,0.1,0.01,0.001,0.0001]
# trade_off=[1000,100,10,1,0.1,0.01]
# best_lr=10
# best_trade_off = 1000
# best_acc = 0
# previous_acc=0

# for i in y_o:
#   for x in trade_off:

      
    
#     epochs = 100
#     w= sub_gradient(train,epochs,i,x)
#     preds = create_predictions(w,test)
#     percentage = accuracy(test, preds)
#     #print(percentage)



#     print(f"Learning rate: {i}, trade-off: {x}, accuracy: {percentage}")

#     if percentage > previous_acc:
#       best_lr = i
#       best_trade_off = x
#       best_acc = percentage
#       previous_acc = percentage

# print(f"The best average cross-validation accuracy was {best_acc} with a learning rate of {best_lr} and a trade off value of {best_trade_off}")

print(f"Testing best parameters....")
#w= sub_gradient(train,1000,best_lr,best_trade_off)
w= sub_gradient(train,1000,0.001,100)
preds = create_predictions(w,test)
percentage = accuracy(test, preds)
print(percentage)
#pd.DataFrame(preds).to_csv("/kaggle/working/testing", header=["example_id"], index=None)


In [ ]:
preds = create_predictions(w,evalTable)

In [ ]:

preds = np.delete(preds,0)
print(preds.shape)
print()
id = '/kaggle/input/uofu-ml-fall-2020/project_data/data/eval.ids'
example = np.genfromtxt(id, delimiter = '\n')
example = example.reshape(preds.shape[0],1)
preds = preds.reshape(preds.shape[0],1)


output = np.concatenate((example, preds), axis = 1).astype(int)
  
pd.DataFrame(output).to_csv("/kaggle/working/bow.eval.subgradient.prediction.csv", header=["example_id", "label"], index=None)